In [1]:
import torch
import random
import numpy as np
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torch.optim.lr_scheduler import MultiStepLR
from utils.util import icarl_cifar100_augment_data, get_dataset_per_pixel_mean

from avalanche.models import IcarlNet, make_icarl_net, initialize_icarl_net
from avalanche.training.supervised import ICaRL
from avalanche.logging import InteractiveLogger, WandBLogger, TensorboardLogger
from avalanche.benchmarks.datasets import CIFAR100
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceTime, EpochAccuracy

/home/cal-05/anaconda3/envs/hspark/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
cudnn.enabled = False # cudnn library를 사용하지 않게 만듬 / cudnn (cuda 라이브러리)의 기본 설정이 시드를 고정하지 않기 때문에 고정된 결과를 얻을 수 있습니다. 그런데 cudnn을 사용하지 않으면 속도가 많이 느려집니다.
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용

In [3]:
train_set = CIFAR100('../Dataset/CIFAR100', train=True, download=True)
test_set = CIFAR100('../Dataset/CIFAR100', train=False, download=True)

pixel_transforms = transforms.Compose([transforms.ToTensor()])
train_per_pixel_mean = get_dataset_per_pixel_mean(CIFAR100('../Dataset/CIFAR100', train=True, download=True, transform=pixel_transforms))
test_per_pixel_mean = get_dataset_per_pixel_mean(CIFAR100('../Dataset/CIFAR100', train=False, download=True, transform=pixel_transforms))

transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              lambda img_pattern: img_pattern - train_per_pixel_mean,
              icarl_cifar100_augment_data,
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              lambda img_pattern: img_pattern - test_per_pixel_mean,
              ]
       ),
       None,
       )
)

train_set = AvalancheDataset(train_set, transform_groups=transforms_group, initial_transform_group="train")
test_set = AvalancheDataset(test_set, transform_groups=transforms_group, initial_transform_group="eval")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 100
incremental = 10
lr_milestones = [49, 63]
lr_factor = 5.0
fixed_class_order = [87, 0, 52, 58, 44, 91, 68, 97, 51, 15, 94, 92, 10, 72, 49, 78, 61, 14, 8, 86, 84, 96, 18, 24, 32, 45, 88, 11, 4, 67, 69, 66, 77, 47, 79, 93, 29, 50, 57, 83, 17, 81, 41, 12, 37, 59, 25, 20, 80, 73, 1, 28, 6, 46, 62, 82, 53, 9, 31, 75, 38, 63, 33, 74, 27, 22, 36, 3, 16, 21, 60, 19, 70, 90, 89, 43, 5, 42, 65, 76, 40, 30, 23, 85, 2, 95, 56, 48, 71, 64, 98, 13, 99, 7, 34, 55, 54, 26, 35, 39]

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=False,
                        fixed_class_order=fixed_class_order
                        )

model: IcarlNet = make_icarl_net(num_classes=num_class)
model.apply(initialize_icarl_net)

optimizer = optim.SGD(model.parameters(), lr=2.0, momentum=0.9, weight_decay=1e-5)
sched = LRSchedulerPlugin(MultiStepLR(optimizer, lr_milestones, gamma=1.0 / lr_factor))

In [5]:
interactive_logger = InteractiveLogger()
# wandb_logger = WandBLogger(project_name="CL-Test", run_name="iCaRL-CIFAR10")
tensor_logger = TensorboardLogger("logs_iCaRL_cifar100")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceTime(),
    # loggers=[interactive_logger, wandb_logger])
    loggers=[interactive_logger, tensor_logger])

/home/cal-05/anaconda3/envs/hspark/lib/python3.8/site-packages/avalanche/training/plugins/evaluation.py:81: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  warnings.warn(


In [6]:
memory_size = 2000
train_batch = 128
eval_batch = 128
epoch = 70

buffer_transform = transforms.Compose([icarl_cifar100_augment_data])

strategies = ICaRL(model.feature_extractor, model.classifier, optimizer, memory_size, buffer_transform=buffer_transform, fixed_memory=True, train_mb_size=train_batch, train_epochs=epoch, eval_mb_size=eval_batch, device=device, plugins=[sched], evaluator=eval_plugin)  # criterion = ICaRLLossPlugin()

In [7]:
for i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i + 1]
    strategies.train(exp)
    strategies.eval(eval_exps)

-- >> Start of training phase << --
100%|██████████| 40/40 [00:08<00:00,  4.48it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2396
100%|██████████| 40/40 [00:08<00:00,  4.49it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4664
100%|██████████| 40/40 [00:08<00:00,  4.46it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5476
100%|██████████| 40/40 [00:08<00:00,  4.51it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5954
100%|██████████| 40/40 [00:08<00:00,  4.47it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6344
100%|██████████| 40/40 [00:08<00:00,  4.49it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6522
100%|██████████| 40/40 [00:08<00:00,  4.47it/s]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.6730
100%|██████████| 40/40 [00:09<00:00,  4.43it/s]
Epoch 7 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7